# 원-핫 인코딩

In [1]:
import re
from konlpy.tag import Okt
from collections import Counter

In [2]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

In [3]:
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")
text = reg.sub('', text)
print(text)

임금님 귀는 당나귀 귀 임금님 귀는 당나귀 귀 실컷 소리치고 나니 속이 확 뚫려 살 것 같았어


In [4]:
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


In [5]:
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [7]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [16]:
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


In [17]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index = word2index[word]
       one_hot_vector[index-1] = 1
       return one_hot_vector

In [18]:
one_hot_encoding('임금님', word2idx)

[0, 1, 0, 0, 0]

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [20]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]

In [21]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력.

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


In [22]:
vocab_size = len(t.word_index) + 1

In [23]:
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


In [24]:
one_hot = to_categorical(encoded, num_classes = vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


# 워드 임베딩(Word Embedding)

## Word2Vec
- 분포 가설 : 비슷한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다
- 중심 단어를 예측하기 위해 윈도우 범위로 주변 단어를 본다.
- 주변 단어, 중심 단어 각각은 원-핫 인코딩 되어 원-핫 벡터가 되고, 원-핫 벡터가 CBoW 나 Skip-gram의 입력이 된다.
1. CBoW (Continuous Bag of words) :
주변에 있는 단어들을 통해 중간에 있는 단어들을 예측
2. Skip-gram : 
중간에 있는 단어로 주변 단어들을 예측하는 방법
- Negative Sampling
    - 기존의 skip-gram은 중심 단어 벡터를 넣어서 모든 단어 V에 대한 예측을 모두 구하여 소프트맥스를 함. 타겟 주변 단어의 원핫벡터와의 차이를 loss로 계산하여 모든 단어에 대한 matrix를 업데이트 해주기 때문에 연산량이 상당하다.
    - 중심단어와 해당 주변 단어를 타겟으로 하는 세트를 만들고, 해당 타겟을 1로 레이블한다. 주변 단어가 아닌 단어를 타겟으로 하는 세트를 만들어서 0로 레이블을 한다. 중심단어와 주변 단어를 곱하고, 활성화함수를 거친 값과 레이블 값의 차이를 loss로 계산하고, 해당 중심단어/주변단어와 관련된 weight만 업데이트 해주기 때문에 연산량을 확 줄였다.

In [28]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /home/aiffel0035/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /home/aiffel0035/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from nltk.corpus import abc
corpus = abc.sents()

In [31]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [32]:
print('코퍼스의 크기 :',len(corpus))

코퍼스의 크기 : 29059


In [37]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)
# size : 학습 후 임베딩 벡터의 차원, min_count : 단어 최소 빈도수 제한 (빈도 적은 단어는 학습하지 않는다.), workers : 학습을 위한 프로세스 수, sg : 0 = CBoW, 1 = Skip-gram

In [38]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9345649480819702), ('Bang', 0.925256609916687), ('third', 0.9196126461029053), ('asteroid', 0.9187909364700317), ('skull', 0.914841890335083), ('rally', 0.913416862487793), ('dinosaur', 0.9078872203826904), ('conviction', 0.9049323201179504), ('flare', 0.9026699066162109), ('clue', 0.9025710225105286)]


In [39]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/word_embedding/w2v') 
loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/word_embedding/w2v")

In [40]:
model_result = loaded_model.wv.most_similar("man")
print(model_result)

[('woman', 0.9345649480819702), ('Bang', 0.925256609916687), ('third', 0.9196126461029053), ('asteroid', 0.9187909364700317), ('skull', 0.914841890335083), ('rally', 0.913416862487793), ('dinosaur', 0.9078872203826904), ('conviction', 0.9049323201179504), ('flare', 0.9026699066162109), ('clue', 0.9025710225105286)]


/home/aiffel0035/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


### 임베딩 벡터의 시각화

``` python
$ python -m gensim.scripts.word2vec2tensor --input ~/aiffel/word_embedding/w2v --output ~/aiffel/word_embedding/w2v
```

[임베딩 프로젝터](https://projector.tensorflow.org/)

## FastText
- 문자 단위 n-gram 표현을 학습한다 (subwords)
- 예) partial (n = 3) : <pa, art, rti, tia, ial, al>, \<partial>
- 예) partial (n_min = 3, n_max = 6) : <pa, art, rti, ita, ial, al>, <par, arti, rtia, tial, ial>, <part, ...중략... , \<partial>
- Subwords 각각에 대해서 Word2Vec을 수행 = 내부 단어들을 벡터화, 최종적으로는 벡터화된 n-gram 벡터들의 총합을 해당 단어의 벡터로 한다.
- (중심단어, 주변 단어)의 쌍을 가지고 이 쌍이 positive인지 negative인지 예측을 진행함. 학습 과정에서 중심 단어에 속한 문자 단위 n-gram 단어 벡터들을 모두 업데이트.

Word2Vec 보다 장점
- OOV와 오타에 강하다.

In [44]:
from gensim.models import FastText
fasttext_model = FastText(corpus, size=100, window=5, min_count=5, workers=4, sg=1)

In [45]:
# Skip-gram 모델에서는 해당 단어가 단어장에 없으면 오류가 난다.
loaded_model.most_similar('overacting')

KeyError: "word 'overacting' not in vocabulary"

In [47]:
# SFastText 모델에서는 해당 단어가 단어장에 없어도 임베딩 벡터값이 계산된다.
fasttext_model.most_similar('overacting')

/home/aiffel0035/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('extracting', 0.9410233497619629),
 ('lifting', 0.9326949119567871),
 ('emptying', 0.9303408861160278),
 ('mixing', 0.9298337697982788),
 ('boosting', 0.9278265237808228),
 ('contracting', 0.9253877997398376),
 ('attracting', 0.9248118996620178),
 ('overwhelming', 0.9241960644721985),
 ('fluctuating', 0.9239733815193176),
 ('hurting', 0.9235039949417114)]

In [48]:
fasttext_model.most_similar('memoryy')

/home/aiffel0035/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('memory', 0.9518458843231201),
 ('musical', 0.8742538690567017),
 ('mechanism', 0.8592822551727295),
 ('intelligence', 0.8482948541641235),
 ('mechanisms', 0.8472954034805298),
 ('technical', 0.8460854887962341),
 ('mechanical', 0.8420599699020386),
 ('practical', 0.8354237079620361),
 ('duplicate', 0.8333635330200195),
 ('specific', 0.8256036639213562)]

### 한국어에서 FastTexst
- 영어가 알파벳 단위의 n-gram이었다면, 한국어는 (1) 음절 단위, (2) 자소 단위 (초성, 중성, 종성 분리) 등으로 할 수 있다.
- 예) 텐서플로우 (n=3) : <텐서, 텐서플, 서플로, 플로우>, <텐서플로우>
- 예) 텐서플로우 (n=3) : <ㅌㅔ,ㅌㅔㄴ,ㅔㄴㅅ,ㄴㅅㅓ,ㅅㅓ_, …중략… >

## GloVe (Global Vectors for Word Representation)
- 윈도우 기반 동시 등장 행렬 : i 단어의 윈도우 크기(window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬
- 동시 등장 확률 P(k|i) : 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률. i = 중심 단어, k = 주변 단어
- 중심 단어 벡터와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 빈도의 로그값이 되도록 학습

In [51]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

from nltk.corpus import movie_reviews
corpus=movie_reviews.sents()

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/aiffel0035/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to /home/aiffel0035/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# glove 패키지는 python3.6까지만 지원
from glove import Corpus, Glove

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
emb = Corpus() 
emb.fit(corpus, window=5)

# 벡터의 차원은 100, 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(emb.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(emb.dictionary)

In [ ]:
model_result = glove.most_similar("man")
print(model_result1)